In [1]:
# Import packages
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
import random

2023-05-07 17:17:58.978855: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def load_larger_dataset_first_scenario():
    dataset = pd.read_csv('signal_1000_posture_4.csv')
    dataset = dataset.reset_index(drop = True)
    dataset['date'] = dataset['Time'].str.slice(stop=10)
    dataset['Day'] = pd.Categorical(dataset.date).codes
    random_values = np.random.randint(0, 44, size=20)
    subset = dataset[dataset['Day'].isin(random_values)]
    total_rows = subset.shape[0]
    train_nrows = int(total_rows * 0.7)

    train = subset.iloc[:train_nrows, :]
    test = subset.iloc[train_nrows: , :]

    rows = train.values.tolist()
    random.shuffle(rows)
    train = pd.DataFrame(data=rows, columns=train.columns)

    rows = test.values.tolist()
    random.shuffle(rows)
    test = pd.DataFrame(data=rows, columns=test.columns)

    train_set_x_orig = train.iloc[:, 2:1002] # train set features
    train_set_y_orig = train.iloc[:, 1] # train set labels

    test_set_x_orig = test.iloc[:, 2:1002]# test set features
    test_set_y_orig = test.iloc[:, 1] # test set labels


    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, dataset, train, test



In [5]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, dataset, train_set, test_set = load_larger_dataset_first_scenario()

In [6]:
pd.DataFrame(train_set_y_orig.value_counts())

,count
Action,
go_to_the_bed,1822
sleep_on_right_side,783
sleep_on_left_side,489
sleep_on_stomach,347


In [7]:
pd.DataFrame(test_set_y_orig.value_counts())

,count
Action,
go_to_the_bed,1179
sleep_on_stomach,113
sleep_on_left_side,97
sleep_on_right_side,87


In [8]:
def transform_label(df):
    mapping = {'go_to_the_bed': 0, 'sleep_on_stomach': 1, 'sleep_on_left_side': 2, 'sleep_on_right_side': 3}
    new_array = np.array(df.replace(mapping))
    return new_array.astype('int64')

In [9]:
train_set_y = transform_label(train_set_y_orig)
test_set_y = transform_label(test_set_y_orig)

Y = pd.get_dummies(train_set_y)
Y = Y.replace({True: 1, False: 0}).astype(float)

from sklearn.preprocessing import StandardScaler

# create a StandardScaler object
scaler = StandardScaler()

# fit the scaler to your dataframe and transform it
train_x_std = scaler.fit_transform(train_set_x_orig)
train_x = pd.DataFrame(train_x_std)

In [64]:
from keras import backend as K
from tensorflow.keras import layers, regularizers
import tensorflow as tf
warnings.filterwarnings('ignore')

def create_first_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(1000),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(2000, activation="relu"),
        tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2000, activation="relu"),
        tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2000, activation="relu"),
        tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2000, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1000, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(200, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(20, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(4, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['acc'])
    return model

In [65]:
import keras
first_model = create_first_model()
callbacks=[
           keras.callbacks.EarlyStopping(monitor='loss',
           patience=5,
           ),
           keras.callbacks.TensorBoard(
               log_dir='my_log_dir',
               histogram_freq=1,
               embeddings_freq=1,
           )
]

first_history = first_model.fit(
    train_x.values,
    Y.values,
    validation_split=0.3,
    epochs=100,
    # callbacks=callbacks,
    batch_size=100,
    callbacks=[callbacks],
);

Epoch 1/100
25/25 [==============================] - 6s 196ms/step - loss: 1.6149 - acc: 0.4215 - val_loss: 4.3874 - val_acc: 0.2227
Epoch 2/100
25/25 [==============================] - 4s 160ms/step - loss: 1.1466 - acc: 0.5714 - val_loss: 3.8255 - val_acc: 0.4124
Epoch 3/100
25/25 [==============================] - 4s 149ms/step - loss: 1.0106 - acc: 0.6154 - val_loss: 3.1016 - val_acc: 0.3020
Epoch 4/100
25/25 [==============================] - 4s 150ms/step - loss: 0.8186 - acc: 0.7139 - val_loss: 2.3135 - val_acc: 0.3795
Epoch 5/100
25/25 [==============================] - 4s 150ms/step - loss: 0.7211 - acc: 0.7546 - val_loss: 1.2800 - val_acc: 0.5808
Epoch 6/100
25/25 [==============================] - 4s 149ms/step - loss: 0.7543 - acc: 0.7521 - val_loss: 1.3354 - val_acc: 0.5799
Epoch 7/100
25/25 [==============================] - 4s 151ms/step - loss: 0.5749 - acc: 0.8119 - val_loss: 1.2224 - val_acc: 0.6409
Epoch 8/100
25/25 [==============================] - 4s 147ms/step - 

In [66]:
scaler = StandardScaler()
# fit the scaler to your dataframe and transform it
test_x_std = scaler.fit_transform(test_set_x_orig)
test_x = pd.DataFrame(test_x_std)

result = first_model.predict(test_x)
tmp = np.argmax(result, axis=1)

from sklearn.metrics import accuracy_score
print(accuracy_score(test_set_y,tmp))

47/47 [==============================] - 1s 12ms/step
0.5074525745257452


In [67]:
from sklearn.metrics import confusion_matrix
NN_confusion_matrix = confusion_matrix(test_set_y, tmp).T
print(NN_confusion_matrix)

[[713  72  66  47]
 [ 97   8  18  15]
 [174   5   4   1]
 [195  28   9  24]]
